In [197]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt','r',encoding='utf-8').read()))

In [287]:
current_context=Counter([])
def find_context():
    norvig_candidates = (known([word]) | known(edits1(word)) | known(edits2(word)) | {word})
    context_candidates = context_sensitive_candidates(context)
    current_context = Counter([w for w in context_candidates if w in norvig_candidates])
    return [w for w in context_candidates if w in norvig_candidates]

def P(word, N=sum(current_context.values())):
    global current_context
    "Probability of `word`."
    print(current_context.values())
    return current_context[word] / N

def correction(word,context): 
    "Most probable spelling correction for word."
    spell_context = Counter(candidates(word,context))
    real_candidates = spell_context.most_common()
#     N=sum(spell_context.values())
#     for i in range(0,len(spell_context)):
#         spell_context[word] = spell_context[word] / N
#     print(spell_context)
    return real_candidates[0]

def candidates(word,context): 
    "Generate possible spelling corrections for word."
    norvig_candidates = (known([word]) | known(edits1(word)) | known(edits2(word)) | {word})
    context_candidates = context_sensitive_candidates(context)
    current_context = Counter([w for w in context_candidates if w in norvig_candidates])
    return [w for w in context_candidates if w in norvig_candidates]

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'آابپتثجچحخدذرزژسشضطظعغفقکگلمنوهی'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [R + L[1:]               for R, L in splits if L]
    transposes = [R + L[1] + L[0] + L[2:] for R, L in splits if len(L)>1]
    replaces   = [R + c + L[1:]           for R, L in splits if L for c in letters]
    inserts    = [R + c + L               for R, L in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [60]:
import pickle
f = open("not_unique_final_cleaned_vocab.pkl", 'rb')
dicti = pickle.load(f)
f.close()
dicti = list(filter(None, dicti))


In [190]:
def sen_corrector(query):
    words = query.split()
    for i in range(0,len(words)):
#         words[i] = correction(words[i],' '.join(words[:i] + words[i+1:]))
        words[i] = correction(words[i],query)
    return words

In [215]:
def context_sensitive_candidates(words):
    possibles = []
    for w in words.split():
        indices = [i for i, x in enumerate(dicti) if x == w]
        for i in indices:
            possibles.extend(dicti[i:i+2] + dicti[i-1:i])
    return possibles

In [288]:
sen_corrector('خیابان اپام')

[('خیابان', 7378), ('امام', 190)]

In [289]:
sen_corrector('کاهن مبعد')

[('کاهن', 141), ('معبد', 9)]